In [32]:
import random
import math
def _alg_ext_euclides(a: int, b: int) -> (int, int, int):
    """
    Argumentos :
        a: int
        b: int - a >= b >= 0 y a > 0
    Retorna :
        (int , int , int) - maximo comun divisor MCD(a, b) entre a y b,
        y numeros enteros s y t tales que MCD(a, b) = s*a + t*b
    """
    r_0 = a
    s_0 = 1
    t_0 = 0
    r_1 = b
    s_1 = 0
    t_1 = 1
    while r_1 > 0:
        r_2 = r_0 % r_1
        s_2 = s_0 - (r_0 // r_1) * s_1
        t_2 = t_0 - (r_0 // r_1) * t_1
        r_0 = r_1
        s_0 = s_1
        t_0 = t_1
        r_1 = r_2
        s_1 = s_2
        t_1 = t_2
    return r_0, s_0, t_0

def _inverso(a: int, n: int) -> int:
    """
    Argumentos :
        a: int - a >= 1
        n: int - n >= 2, a y n son primos relativos
    Retorna :
        int - inverso de a en modulo n
    """
    (r, s, t) = _alg_ext_euclides(a, n)
    return s % n

def _exp(a: int, b: int) -> int:
    """
    Argumentos :
        a: int
        b: int - b >= 0
    Retorna :
        int - a**b
    """
    if b == 0:
        return 1
    else:
        res = 1
        pot = a
        while b > 0:
            if b % 2 == 1:
                res = pot * res
            b = b // 2
            pot = pot * pot
        return res

def _tiene_raiz_entera_intervalo(n: int, k: int, i: int, j: int) -> bool:
    """
    Argumentos :
        n: int - n >= 1
        k: int - k >= 2
        i: int - i >= 0
        j: int - j >= 0
    Retorna :
        bool - True si existe numero natural a tal que n = (a**k),
        donde i <= a <= j. En caso contrario retorna False.       
    """
    while i <= j:
        if i==j:
            return n == _exp(i,k)
        else:
            p = (i + j)//2 
            val = _exp(p,k)
            if n == val:
                return True
            elif val < n:
                i = p+1
            else:
                j = p-1
    return False

def _tiene_raiz_entera(n: int, k: int) -> bool:
    """
    Argumentos :
        n: int - n >= 1
        k: int - k >= 2
    Retorna :
        bool - True si existe numero natural a tal que n = (a**k),
        donde a >= 2. En caso contrario retorna False.       
    """
    if n <= 3:
        return False
    else:
        a = 1
        while _exp(a,k) < n:
            a = 2*a
        return _tiene_raiz_entera_intervalo(n, k, a//2, a)

def _mcd(a: int, b: int) -> int:
    """
    Argumentos :
        a: int
        b: int - a > 0 o b > 0
    Retorna :
        maximo comun divisor entre a y b,
    """
    while b > 0:
        temp = b
        b = a % b
        a = temp
    return a

def _es_potencia(n: int) -> bool:
    """
    Argumentos :
        n: int - n >= 1
    Retorna :
        bool - True si existen numeros naturales a y b tales que n = (a**b),
        donde a >= 2 y b >= 2. En caso contrario retorna False.       
    """
    if n <= 3:
        return False
    else:
        k = 2
        lim = 4
        while lim <= n:
            if _tiene_raiz_entera(n, k):
                return True
            k = k + 1
            lim = lim * 2
        return False

def _exp_mod(a: int, b: int, n: int) -> int:
    """
    Argumentos :
        a: int
        b: int
        n: int - n > 0
    Retorna :
        int - a**b en modulo n
    """
    if b == 0:
        return 1
    elif b > 0:
        res = 1
        pot = a
        while b > 0:
            if b % 2 == 1:
                res = (pot * res) % n
            b = b // 2
            pot = (pot * pot) % n
        return res
    else:
        return _exp_mod(_inverso(a,n),-b,n)

def _test_primalidad(n: int, k: int) -> bool:
    """
    Argumentos :
        n: int - n >= 1
        k: int - k >= 1
    Retorna :
        bool - True si n es un numero primo, y False en caso contrario.
        La probabilidad de error del test es menor o igual a 2**(-k),
        y esta basado en el test de primalidad de Solovay–Strassen
    """
    if n == 1:
        return False
    elif n == 2:
        return True
    elif n%2 == 0:
        return False
    # elif es_potencia(n):
    #     return False
    else:
        neg = 0
        for i in range(1,k+1):
            a = random.randint(2,n-1)
            if _mcd(a,n) > 1:
                return False
            else:
                b = _exp_mod(a,(n-1)//2,n)
                if b == n - 1:
                    neg = neg + 1
                elif b != 1:
                    return False
        if neg > 0:
            return True
        else:
            return False

def _buscar_primo(a,b):
    maximo = (b - a)//2 + 1
    probados = []
    if a%2 == 0:
        step = 1
    else:
        step = 2
    while len(probados) < maximo: 
        n = random.randrange(a,b,step)
        if (n not in probados):
            if _test_primalidad(n, 40):
                return n
            probados.append(n)
    return -1

def _int_to_bytes(x: int) -> bytes:
    return x.to_bytes((x.bit_length() + 7) // 8, 'big')

In [33]:
class RSAReceiver:
  def __init__(self, bit_len: int) -> None:
    self.bit_len = bit_len
    self.bit_len_primo = math.ceil(bit_len/2)
    self._generar_key()


  def _generar_key(self):
    min_prim = 2**(self.bit_len_primo)
    max_prim = 2**(self.bit_len_primo + 7)
    P = _buscar_primo(min_prim, max_prim)
    Q = _buscar_primo(min_prim, max_prim)
    while P == Q:
      Q = _buscar_primo(min_prim, max_prim)
    N = P*Q

    ## generar d
    ## MCD(d, (P-1)*(Q-1)) = 1 Es decir sean primos relativos
    phi_N = (P-1)*(Q-1)
    d = random.randint(0,phi_N)
    while _mcd(d,phi_N)!=1:
      d = random.randint(0,phi_N)
    
    # generar e, el inverso modular
    e = _inverso(d,phi_N)

    self.private_key = (d, N)
    self.public_key = (e, N)
  
  def get_public_key(self) -> bytearray:
    e = self.public_key[0]
    N = self.public_key[1]
    bytes_e = bytearray(_int_to_bytes(e))
    bytes_N = bytearray(_int_to_bytes(N))
    len_e = len(bytes_e).to_bytes(4, 'big')
    len_N = len(bytes_N).to_bytes(4, 'big')
    return len_e + bytes_e + len_N + bytes_N
    

  def decrypt(self, ciphertext: bytearray) -> str:
    # obtener e de la clave publica
    e = self.public_key[0]
    # obtener N de la clave publica
    N = self.public_key[1]
    # obtener d de la clave privada
    d = self.private_key[0]

    # m = c**d mod N

    # calcular el tamaño del bloque
    n_block = (N.bit_length() // 8) + 1

    iter = math.ceil(len(ciphertext)/n_block)
    m = bytearray(b'')
    for i in range(iter):
      block = ciphertext[i*n_block:(i+1)*n_block]
      a = pow(int.from_bytes(block, 'big'), d, N)
      m.extend(a.to_bytes(n_block, 'big'))

    return m.decode('utf-8').replace('\x00', '')



In [34]:
class RSASender:
  def __init__(self, public_key: bytearray) -> None:
    self.public_key = public_key
  
  def encrypt(self, message: str) -> bytearray:
    # obtener e de la clave publica
    len_e = self.public_key[:4]
    len_e = int.from_bytes(len_e, "big")
    e = int.from_bytes(self.public_key[4:len_e+4], "big")
    # obtener N de la clave publica
    len_N = self.public_key[len_e + 4:len_e + 8]
    len_N = int.from_bytes(len_N, "big")
    N = int.from_bytes(self.public_key[len_e + 8:len_e + 8 + len_N], "big")

    # calcular el tamaño del bloque
    n_block = N.bit_length() // 8
    if n_block*8 >= N.bit_length():
      n_block -= 1

    m = bytearray(message, 'utf-8')
    iter = math.ceil(len(m)/n_block)
    c = bytearray(b'')
    for i in range(iter):
      block = m[i*n_block:(i+1)*n_block]
      a = pow(int.from_bytes(block, 'big'), e, N)
      c.extend(a.to_bytes(n_block+1, 'big'))

    return c